In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
MACD + Fibonacci Retracement Swing Strategy (vectorbt) — No CLI
===============================================================

• All parameters are set in the CONFIG block below (list of tickers supported).
• Prints every trade taken.
• Saves trades.csv and summary.csv into ./outputs/<timestamp>/

Requires:
    pip install vectorbt yfinance pandas numpy
"""

from __future__ import annotations
import os
import sys
import math
import warnings
from dataclasses import dataclass, asdict
from datetime import datetime, timezone
from typing import List, Tuple

import numpy as np
import pandas as pd

# -------------------------
# CONFIG — Edit here
# -------------------------
SYMBOLS: List[str] = ["RELIANCE.NS", "SBIN.NS", "INFY.NS"]  # ← list of tickers
START: str = "2019-01-01"
END: str   = "2025-01-01"
TIMEFRAME: str = "1d"                 # one of: "1h", "4h", "1d"
TIMEZONE: str  = "Asia/Kolkata"

# Indicators
SMA_LEN_TREND: int = 50
SMA_SLOPE_LOOKBACK: int = 5
MACD_FAST: int = 12
MACD_SLOW: int = 26
MACD_SIGNAL: int = 9

# Pivots / Fibonacci
PIVOT_LEFT: int = 5
PIVOT_RIGHT: int = 5
FIB_LEVELS: Tuple[float, ...] = (0.382, 0.5, 0.618)
FIB_TOLERANCE: float = 0.003       # 0.3% tolerance around Fib price

# Risk & costs
INIT_CASH: float = 2_00_000.0      # ₹2 lakh
RISK_PER_TRADE: float = 0.01       # 1% of equity
SLIPPAGE: float = 0.0005           # 5 bps per trade
FEES_PCT: float = 0.0008           # 8 bps per side approx

# Misc
OUTDIR: str = "./outputs"
SEED: int = 42


# -------------------------
# Imports that may be optional in this environment
# -------------------------
try:
    import vectorbt as vbt
    VBT_OK = True
except Exception as e:
    VBT_OK = False
    print("Warning: vectorbt not available in this environment:", e, file=sys.stderr)

try:
    import yfinance as yf
    YF_OK = True
except Exception as e:
    YF_OK = False
    print("Warning: yfinance not available in this environment:", e, file=sys.stderr)


# -------------------------
# Data structures
# -------------------------
@dataclass
class Config:
    symbols: List[str]
    start: str
    end: str
    timeframe: str
    tz: str
    sma_len_trend: int
    sma_slope_lookback: int
    macd_fast: int
    macd_slow: int
    macd_signal: int
    pivot_left: int
    pivot_right: int
    fib_levels: Tuple[float, ...]
    fib_tolerance: float
    init_cash: float
    risk_per_trade: float
    slippage: float
    fees_pct: float
    outdir: str
    seed: int


def build_config_from_globals() -> Config:
    return Config(
        symbols=SYMBOLS,
        start=START,
        end=END,
        timeframe=TIMEFRAME,
        tz=TIMEZONE,
        sma_len_trend=SMA_LEN_TREND,
        sma_slope_lookback=SMA_SLOPE_LOOKBACK,
        macd_fast=MACD_FAST,
        macd_slow=MACD_SLOW,
        macd_signal=MACD_SIGNAL,
        pivot_left=PIVOT_LEFT,
        pivot_right=PIVOT_RIGHT,
        fib_levels=FIB_LEVELS,
        fib_tolerance=FIB_TOLERANCE,
        init_cash=INIT_CASH,
        risk_per_trade=RISK_PER_TRADE,
        slippage=SLIPPAGE,
        fees_pct=FEES_PCT,
        outdir=OUTDIR,
        seed=SEED
    )


# -------------------------
# Helpers
# -------------------------
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


def date_dir(base: str) -> str:
    d = datetime.now(timezone.utc).astimezone().strftime("%Y-%m-%d_%H%M%S")
    path = os.path.join(base, d)
    ensure_dir(path)
    return path


def to_tz(df: pd.DataFrame, tz: str) -> pd.DataFrame:
    if df.index.tz is None:
        return df.tz_localize("UTC").tz_convert(tz)
    return df.tz_convert(tz)


def fetch_prices(symbols: List[str], start: str, end: str, timeframe: str, tz: str, seed: int) -> pd.DataFrame:
    """Fetch OHLCV via yfinance; if unavailable, generate synthetic GBM data."""
    if YF_OK:
        interval = {"1h": "60m", "4h": "240m", "1d": "1d"}.get(timeframe, "60m")
        data = {}
        for s in symbols:
            try:
                df = yf.download(s, start=start, end=end, interval=interval, auto_adjust=False, progress=False, multi_level_index=False)
                if df.empty:
                    raise RuntimeError(f"No data for {s}")
                df = df.rename(columns=str.title)  # 'Open','High','Low','Close','Volume'
                data[s] = df[["Open","High","Low","Close","Volume"]]
            except Exception as e:
                print(f"yfinance failed for {s}: {e}. Falling back to synthetic.", file=sys.stderr)
                data[s] = make_synth_ohlc(start, end, timeframe, seed=seed+hash(s)%9973)
        closes = pd.concat({s: d["Close"] for s, d in data.items()}, axis=1).dropna(how="all")
        out = {}
        for field in ["Open","High","Low","Close","Volume"]:
            out[field] = pd.concat({s: d[field].reindex(closes.index).ffill() for s, d in data.items()}, axis=1)
        ohlc = pd.concat(out, axis=1)
        ohlc.index = pd.to_datetime(ohlc.index)
        ohlc = to_tz(ohlc, tz)
        return ohlc
    else:
        print("Using synthetic OHLC for all symbols.", file=sys.stderr)
        out = {}
        for s in symbols:
            out[s] = make_synth_ohlc(start, end, timeframe, seed=seed+hash(s)%9973)
        fields = {}
        for field in ["Open","High","Low","Close","Volume"]:
            fields[field] = pd.concat({s: d[field] for s, d in out.items()}, axis=1)
        ohlc = pd.concat(fields, axis=1)
        ohlc.index = pd.to_datetime(ohlc.index)
        ohlc = to_tz(ohlc, tz)
        return ohlc


def make_synth_ohlc(start: str, end: str, timeframe: str, seed: int = 42) -> pd.DataFrame:
    """Simple GBM synthetic OHLC so script runs end-to-end without internet."""
    rng = np.random.default_rng(seed)
    start_ts = pd.Timestamp(start)
    end_ts = pd.Timestamp(end)
    if timeframe == "1h":
        idx = pd.date_range(start_ts, end_ts, freq="1H")
    elif timeframe == "4h":
        idx = pd.date_range(start_ts, end_ts, freq="4H")
    else:
        idx = pd.date_range(start_ts, end_ts, freq="1D")

    n = len(idx)
    drift = 0.0001
    vol = 0.01 if timeframe != "1d" else 0.02
    returns = rng.normal(drift, vol, size=n)
    price = 100 * np.exp(np.cumsum(returns))
    close = pd.Series(price, index=idx, name="Close")
    high = close * (1 + rng.uniform(0, 0.003, size=n))
    low  = close * (1 - rng.uniform(0, 0.003, size=n))
    open_ = close.shift(1).fillna(close.iloc[0])
    volm = pd.Series(rng.integers(1e5, 2e6, size=n), index=idx, name="Volume").astype(float)
    df = pd.DataFrame({"Open": open_, "High": high, "Low": low, "Close": close, "Volume": volm})
    return df


def rolling_pivots(series: pd.Series, left: int, right: int) -> tuple[pd.Series, pd.Series]:
    """Return pivot_high and pivot_low boolean series using a simple rolling window approach."""
    highs = series
    lows  = series
    ph = highs[(highs.rolling(left+1).max() == highs) & (highs.shift(-right).rolling(right+1).max() == highs)]
    pl = lows[(lows.rolling(left+1).min() == lows) & (lows.shift(-right).rolling(right+1).min() == lows)]
    pivot_high = series.index.isin(ph.index)
    pivot_low  = series.index.isin(pl.index)
    return pd.Series(pivot_high, index=series.index), pd.Series(pivot_low, index=series.index)


def last_swing_leg(high: pd.Series, low: pd.Series, trend_up: pd.Series, trend_down: pd.Series,
                   left: int, right: int) -> pd.DataFrame:
    """For each bar, find the most recent swing leg (low->high for uptrend, high->low for downtrend)."""
    ph, pl = rolling_pivots(high, left, right)
    ph_idx = high.index[ph]
    pl_idx = low.index[pl]

    swing_high = pd.Series(np.nan, index=high.index, dtype=float)
    swing_low  = pd.Series(np.nan, index=low.index, dtype=float)

    last_high = np.nan
    last_low  = np.nan
    for i, ts in enumerate(high.index):
        if ph.loc[ts]:
            last_high = high.loc[ts]
        if pl.loc[ts]:
            last_low  = low.loc[ts]
        swing_high.iloc[i] = last_high
        swing_low.iloc[i]  = last_low

    df = pd.DataFrame({
        "swing_high": swing_high,
        "swing_low": swing_low,
        "trend_up": trend_up.astype(bool),
        "trend_down": trend_down.astype(bool)
    }, index=high.index)

    df["leg_ok_up"] = (df["swing_high"] > df["swing_low"])
    df["leg_ok_dn"] = (df["swing_high"] > df["swing_low"])

    return df


def compute_fib_levels(low_val: float, high_val: float) -> dict:
    """Fibonacci retracement levels for an uptrend leg (low→high)."""
    if pd.isna(low_val) or pd.isna(high_val) or high_val <= low_val:
        return {}
    diff = high_val - low_val
    levels = {
        0.0: high_val,     # Target
        0.382: high_val - 0.382 * diff,
        0.5:   high_val - 0.5 * diff,
        0.618: high_val - 0.618 * diff,
        1.0:  low_val
    }
    return levels


def compute_fib_levels_down(high_val: float, low_val: float) -> dict:
    """Fibonacci retracement levels for a downtrend leg (high→low)."""
    if pd.isna(low_val) or pd.isna(high_val) or high_val <= low_val:
        return {}
    diff = high_val - low_val
    levels = {
        0.0: low_val,      # Target
        0.382: low_val + 0.382 * diff,
        0.5:  low_val + 0.5 * diff,
        0.618: low_val + 0.618 * diff,
        1.0:  high_val
    }
    return levels


def slope(series: pd.Series, lookback: int) -> pd.Series:
    """Simple slope proxy: series.diff(lookback)."""
    return series.diff(lookback)


def generate_signals(ohlc: pd.DataFrame, cfg: Config):
    """Build entry signals, per-symbol stop/target ratios, position sizes (risk-based)."""
    if not VBT_OK:
        raise RuntimeError("vectorbt is required to build signals/backtest. Please install vectorbt.")

    close = ohlc["Close"]
    high  = ohlc["High"]
    low   = ohlc["Low"]

    # Trend filter on daily resampled prices (for 1h/4h data)
    daily_close = close.resample("1D").last().ffill()
    sma = daily_close.vbt.ma(cfg.sma_len_trend)
    up_slope = slope(sma, cfg.sma_slope_lookback) > 0
    dn_slope = slope(sma, cfg.sma_slope_lookback) < 0
    daily_up   = (daily_close > sma) & up_slope
    daily_down = (daily_close < sma) & dn_slope

    intr_idx = close.index
    trend_up   = daily_up.reindex(intr_idx, method="ffill")
    trend_down = daily_down.reindex(intr_idx, method="ffill")

    # MACD on intraday
    macd = vbt.MACD.run(close, fast_window=cfg.macd_fast, slow_window=cfg.macd_slow, signal_window=cfg.macd_signal)
    macd_line = macd.macd
    macd_signal = macd.signal
    macd_cross_up = macd_line.vbt.crossed_above(macd_signal)
    macd_cross_dn = macd_line.vbt.crossed_below(macd_signal)

    entries_long = close.iloc[0:0].copy()
    entries_short = close.iloc[0:0].copy()

    sl_ratio = pd.DataFrame(0.0, index=close.index, columns=close.columns)
    tp_ratio = pd.DataFrame(0.0, index=close.index, columns=close.columns)
    size = pd.DataFrame(0.0, index=close.index, columns=close.columns)

    for sym in close.columns:
        c = close[sym]
        h = high[sym]
        l = low[sym]
        tu = trend_up[sym].fillna(False)
        td = trend_down[sym].fillna(False)

        swings = last_swing_leg(h, l, tu, td, cfg.pivot_left, cfg.pivot_right)

        long_mask = pd.Series(False, index=close.index)
        short_mask = pd.Series(False, index=close.index)

        for t in range(len(c)):
            ts = c.index[t]
            if pd.isna(swings.loc[ts, "swing_high"]) or pd.isna(swings.loc[ts, "swing_low"]):
                continue

            # LONG
            if tu.loc[ts] and swings.loc[ts, "leg_ok_up"]:
                fibs = compute_fib_levels(swings.loc[ts, "swing_low"], swings.loc[ts, "swing_high"])
                if fibs:
                    price = c.loc[ts]
                    touched = None
                    for lvl in cfg.fib_levels:
                        if lvl in fibs and abs(price - fibs[lvl]) / fibs[lvl] <= cfg.fib_tolerance:
                            touched = lvl
                            break
                    if touched is not None and macd_cross_up[sym].loc[ts]:
                        long_mask.loc[ts] = True
                        # SL / TP
                        if touched == 0.382:
                            stop_price = fibs[0.5]
                        elif touched == 0.5:
                            stop_price = fibs[0.618]
                        else:
                            stop_price = fibs[1.0]
                        target_price = fibs[0.0]

                        entry_price = price
                        sl = (entry_price - stop_price) / entry_price if (pd.notna(stop_price) and stop_price < entry_price) else 0.02
                        tp = (target_price - entry_price) / entry_price if (pd.notna(target_price) and target_price > entry_price) else 0.04

                        sl_ratio.at[ts, sym] = sl
                        tp_ratio.at[ts, sym] = tp

                        risk_amt = cfg.init_cash * cfg.risk_per_trade
                        shares = math.floor(risk_amt / (sl * entry_price)) if sl > 0 else 0
                        size.at[ts, sym] = max(shares, 0)

            # SHORT
            if td.loc[ts] and swings.loc[ts, "leg_ok_dn"]:
                fibs_dn = compute_fib_levels_down(swings.loc[ts, "swing_high"], swings.loc[ts, "swing_low"])
                if fibs_dn:
                    price = c.loc[ts]
                    touched = None
                    for lvl in cfg.fib_levels:
                        if lvl in fibs_dn and abs(price - fibs_dn[lvl]) / fibs_dn[lvl] <= cfg.fib_tolerance:
                            touched = lvl
                            break
                    if touched is not None and macd_cross_dn[sym].loc[ts]:
                        short_mask.loc[ts] = True

                        if touched == 0.382:
                            stop_price = fibs_dn[0.5]
                        elif touched == 0.5:
                            stop_price = fibs_dn[0.618]
                        else:
                            stop_price = fibs_dn[1.0]
                        target_price = fibs_dn[0.0]

                        entry_price = price
                        sl = (stop_price - entry_price) / entry_price if (pd.notna(stop_price) and stop_price > entry_price) else 0.02
                        tp = (entry_price - target_price) / entry_price if (pd.notna(target_price) and target_price < entry_price) else 0.04

                        sl_ratio.at[ts, sym] = sl
                        tp_ratio.at[ts, sym] = tp

                        risk_amt = cfg.init_cash * cfg.risk_per_trade
                        shares = math.floor(risk_amt / (sl * entry_price)) if sl > 0 else 0
                        size.at[ts, sym] = max(shares, 0)

        entries_long = pd.concat([entries_long, long_mask.to_frame(sym)], axis=1) if not entries_long.empty else long_mask.to_frame(sym)
        entries_short = pd.concat([entries_short, short_mask.to_frame(sym)], axis=1) if not entries_short.empty else short_mask.to_frame(sym)

    entries_long = entries_long.reindex(close.index).fillna(False)
    entries_short = entries_short.reindex(close.index).fillna(False)

    meta = {
        "sl_ratio": sl_ratio.where(entries_long | entries_short, other=0.0),
        "tp_ratio": tp_ratio.where(entries_long | entries_short, other=0.0),
        "size": size.where(entries_long | entries_short, other=0.0),
        "entries_long": entries_long,
        "entries_short": entries_short,
        "trend_up": trend_up,
        "trend_down": trend_down
    }
    return entries_long, entries_short, meta


def build_portfolio(ohlc: pd.DataFrame, entries_long: pd.DataFrame, entries_short: pd.DataFrame, meta: dict, cfg: Config):
    """Build vectorbt Portfolio using from_signals with per-entry stop/target and signed sizes."""
    if not VBT_OK:
        raise RuntimeError("vectorbt is required to build portfolio. Please install vectorbt.")

    close = ohlc["Close"]
    sl_ratio = meta["sl_ratio"]
    tp_ratio = meta["tp_ratio"]
    sizes = meta["size"]

    long_sizes = sizes.where(entries_long, other=0.0)
    short_sizes = -sizes.where(entries_short, other=0.0)  # negative => short
    side_sizes = (long_sizes + short_sizes)

    pf = vbt.Portfolio.from_signals(
        close,
        entries=(entries_long | entries_short),
        exits=None,
        init_cash=cfg.init_cash,
        size=side_sizes,
        sl_stop=sl_ratio,
        tp_stop=tp_ratio,
        fees=cfg.fees_pct,
        slippage=cfg.slippage,
        direction="both"
    )
    return pf


def readable_trades(pf, ohlc: pd.DataFrame) -> pd.DataFrame:
    """Return human-readable trades (vectorbt's records_readable)."""
    if pf.trades.count() == 0:
        return pd.DataFrame()
    return pf.trades.records_readable


def summarize_portfolio(pf) -> pd.DataFrame:
    cols = []
    for col in pf.wrapper.columns:
        s = {
            "symbol": col,
            "trades": int(pf.trades.count(col=col)),
            "win_rate_%": round(float(pf.trades.win_rate(col=col))*100, 2) if pf.trades.count(col=col) > 0 else 0.0,
            "total_pnl": float(pf.trades.pnl_sum(col=col)),
            "total_return_%": round(float(pf.total_return(col=col))*100, 2) if pf.trades.count(col=col) > 0 else 0.0,
            "max_dd_%": round(float(pf.max_drawdown(col=col))*100, 2) if pf.trades.count(col=col) > 0 else 0.0,
            "sharpe": round(float(pf.sharpe_ratio(col=col)), 2) if pf.trades.count(col=col) > 0 else np.nan
        }
        cols.append(s)
    return pd.DataFrame(cols)


def print_trades(trades_df: pd.DataFrame, max_rows: int = 100):
    if trades_df.empty:
        print("\nNo trades were taken.\n")
        return
    print("\n=== TRADES TAKEN (first {} shown) ===".format(max_rows))
    show = trades_df.head(max_rows)
    for _, r in show.iterrows():
        sym = r.get("Column", "")
        side = r.get("Direction", "")
        et  = r.get("Entry Timestamp", "")
        ep  = r.get("Entry Price", np.nan)
        xt  = r.get("Exit Timestamp", "")
        xp  = r.get("Exit Price", np.nan)
        sz  = r.get("Position Size", np.nan)
        pnl = r.get("PnL", np.nan)
        ret = r.get("Return [%]", np.nan)
        print(f"{sym:>12} | {side:>5} | in: {et} @ {ep:.2f} | out: {xt} @ {xp:.2f} | size: {sz:.2f} | PnL: {pnl:.2f} | Return%: {ret:.2f}")


def run(cfg: Config):
    print("CONFIG:", asdict(cfg))
    ohlc = fetch_prices(cfg.symbols, cfg.start, cfg.end, cfg.timeframe, cfg.tz, cfg.seed)
    if not VBT_OK:
        raise RuntimeError("vectorbt is required. Install with: pip install vectorbt")

    entries_long, entries_short, meta = generate_signals(ohlc, cfg)
    pf = build_portfolio(ohlc, entries_long, entries_short, meta, cfg)

    trades_df = readable_trades(pf, ohlc)
    summary_df = summarize_portfolio(pf)

    outdir = date_dir(cfg.outdir)
    trades_path = os.path.join(outdir, "trades.csv")
    summary_path = os.path.join(outdir, "summary.csv")
    trades_df.to_csv(trades_path, index=False)
    summary_df.to_csv(summary_path, index=False)

    print_trades(trades_df, max_rows=100)
    print("\n=== SUMMARY ===")
    print(summary_df.to_string(index=False))

    print("\n=== PORTFOLIO METRICS ===")
    print(f"Total Return: {pf.total_return():.2%}")
    print(f"Win Rate: {pf.trades.win_rate():.2%}")
    print(f"Max Drawdown: {pf.max_drawdown():.2%}")
    print(f"Sharpe: {pf.sharpe_ratio():.2f}")
    print(f"\nCSV saved to: {outdir}")


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    cfg = build_config_from_globals()
    run(cfg)


CONFIG: {'symbols': ['RELIANCE.NS', 'SBIN.NS', 'INFY.NS'], 'start': '2019-01-01', 'end': '2025-01-01', 'timeframe': '1d', 'tz': 'Asia/Kolkata', 'sma_len_trend': 50, 'sma_slope_lookback': 5, 'macd_fast': 12, 'macd_slow': 26, 'macd_signal': 9, 'pivot_left': 5, 'pivot_right': 5, 'fib_levels': (0.382, 0.5, 0.618), 'fib_tolerance': 0.003, 'init_cash': 200000.0, 'risk_per_trade': 0.01, 'slippage': 0.0005, 'fees_pct': 0.0008, 'outdir': './outputs', 'seed': 42}


AttributeError: 'Vbt_DFAccessor' object has no attribute 'ma'